<a href="https://colab.research.google.com/github/deepskies/SimplePhysicalImage/blob/master/Lensing_Simulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Wedge, Rectangle
from matplotlib.path import Path
from bokeh.plotting import figure, show, output_notebook, gridplot

# Create Methods for Structure Creation

In [2]:
def _patch2im(thepatch, imshape):
    # Create meshgrid of coordinates
    y = np.arange(1, imshape[0])
    x = np.arange(1, imshape[1])
    g = np.meshgrid(x, y)

    coords = np.array(list(zip(*(c.flat for c in g))))

    # Create path
    elpath = Path(thepatch.get_verts())

    # Determine all points in the grid ithat are in the path.
    validcoords = elpath.contains_points(coords)
    ellipsepoints = coords[validcoords]

    outim = np.zeros(imshape)
    outim[ellipsepoints[:, 0], ellipsepoints[:, 1]] = 1

    return outim


def create_polygon(imshape, xy, width, height, angle=0.0, fill=True):

    # Create the arc patch we want to add to the image
    rect = Rectangle(xy, width, height, angle=0.0, fill=fill)

    return _patch2im(rect, imshape)


def create_circle(imshape, center, radius, width):
    """
    Create a cirlce.
    """

    # Create the arc patch we want to add to the image
    return create_arc(imshape, center, radius, width=width, theta1=0, theta2=360)
  
def create_line(imshape, center, radius, width):
    """
    Create a cirlce.
    """

    # Create the arc patch we want to add to the image
    # return create_line(imshape, center, radius, width=width, theta1=0, theta2=359)


def create_arc(imshape, center, radius, width, theta1, theta2):
    """
    Create an arc (outline of a circle) with a specified
    thickness (width).
    """

    # Create the arc patch we want to add to the image
    arc = Wedge(center, radius, theta1, theta2, width)

    return _patch2im(arc, imshape)

def create_noise(imshape, noise_level):
    """
    Create noise image
    https://kmdouglass.github.io/posts/modeling-noise-for-image-simulations.html
    """
    
    seed = 40
    rs = np.random.RandomState(seed)
    
    #return rs.poisson(noise_level, (np.shape(imshape)[0],np.shape(imshape)[1]))
    return rs.poisson(noise_level, size=imshape)

def create_noise_1(imshape, noise_level, seed):
    """
    Create noise image
    https://kmdouglass.github.io/posts/modeling-noise-for-image-simulations.html
    """
    
    #seed = 40
    rs = np.random.RandomState(seed)
    
    #return rs.poisson(noise_level, (np.shape(imshape)[0],np.shape(imshape)[1]))
    return rs.poisson(noise_level, size=imshape)

# Create an Example Lensing Image

In [21]:
imblah = np.zeros((256, 256))

# Example arc
arc = create_arc(imblah.shape, center=(150, 128), radius=56, width=6, theta1=27, theta2=156)
imblah += arc * 35

# Example filled circle
circle = create_circle(imblah.shape, center=(50, 28), radius=56, width=56)
imblah += circle * 21

# Example unfilled circle
circle = create_circle(imblah.shape, center=(28, 200), radius=50, width=8)
imblah += circle * 61

# Example polygon
rect = create_polygon(imblah.shape, (150, 28), width=27, height=44)
imblah += rect * 13

In [57]:
imblah = np.zeros((100, 100,1))

# Example arc
circle = create_circle(imblah.shape, center=(50, 65), radius=20, width=6)
imblah += circle * 60

noise = create_noise_1(imblah.shape, noise_level=100.)
imblah += noise

# Make Image

In [58]:
p = figure(x_range=(1, 100), y_range=(1, 100))

# must give a vector of image data for image parameter
p.image(image=[imblah], x=1, y=1, dw=100, dh=100, palette="Spectral11")

output_notebook()

show(p)  # open a browser

Loading BokehJS ...

# Make image samples

In [20]:
figures = []
figures_up = []
figures_down = []
nb_sample = 1000

#main sample
for ii in np.arange(nb_sample):
    image = np.zeros((100, 100,1))
    circle = create_circle(image.shape, center=(50, 65), radius=20, width=6)
    image += circle * 60

    #the only difference between images is the noise
    noise = create_noise_1(image.shape, noise_level=100., seed=ii)
    image += noise
    figures.append(image) 

    
#derivative up
for ii in np.arange(nb_sample):
    image = np.zeros((100, 100,1))
    circle = create_circle(image.shape, center=(50, 65), radius=21, width=6)
    image += circle * 60

    #the only difference between images is the noise
    noise = create_noise_1(image.shape, noise_level=100., seed=ii)
    image += noise
    figures_up.append(image) 
    

#derivative down
for ii in np.arange(nb_sample):
    image = np.zeros((100, 100,1))
    circle = create_circle(image.shape, center=(50, 65), radius=19, width=6)
    image += circle * 60

    #the only difference between images is the noise
    noise = create_noise_1(image.shape, noise_level=100., seed=ii)
    image += noise
    figures_down.append(image) 

In [23]:
np.shape(figures_down)

(1000, 100, 100, 1)

In [26]:
p = figure(x_range=(1, 100), y_range=(1, 100))

# must give a vector of image data for image parameter
p.image(image=[figures_up[0]], x=1, y=1, dw=100, dh=100, palette="Spectral11")

output_notebook()

show(p)  # open a brow

Loading BokehJS ...

In [27]:
np.save("circles.npy",figures)
np.save("circles_up.npy",figures_up)
np.save("circles_down.npy",figures_down)